In [20]:
#import libraries
import sys
from os.path import dirname, join as pjoin
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.linalg as la
from statistics import mean 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
np.set_printoptions(threshold=1000)

In [21]:
#load heart data into arr
heart_data_dir= "heart.mat"
heart_data = sio.loadmat(heart_data_dir)
arr = heart_data['dat']
print(arr.shape)

(270, 13)


In [22]:
#load the labels describing presence or absence of disease
labels = heart_data['label']
print(labels.shape)

(270, 1)


In [23]:
def featureSubsetScore(arr, labels):   
    # instantiate the model
    logreg = LogisticRegression(solver="liblinear", C=0.303)
    trainScores = [0]*10
    testScores = [0]*10
    nTrain = 200
    nTest = 70
    #repeating 10 times inside for loop
    for i in range(0,10):
        #Resetting the default random number generator with a new random seed 
        random.seed(i+1)
        
        #initialize randomMatrix from original matrix
        randomArr = arr.copy()
        randomLabels = labels.copy();

        #randomize by swapping rows in random Matrix's randomArr and randomLabels
        randomIndices = list(range(0,270))
        random.shuffle(randomIndices)
        for j in range(0,270):            
            temp = randomArr[j]
            randomArr[j] = randomArr[randomIndices[j]]
            randomArr[randomIndices[j]] = temp
            
            temp = randomLabels[j]
            randomLabels[j] = randomLabels[randomIndices[j]]
            randomLabels[randomIndices[j]] = temp
        #First 200 for training    
        trainingArr = randomArr[range(0,nTrain),:]     
        trainingLabels = randomLabels[range(0,nTrain)]
        
        #remaining 70 for testing
        testingArr = randomArr[range(nTrain, nTrain+nTest),:]    
        testingLabels = randomLabels[range(nTrain, nTrain+nTest)].ravel()
        
        #Training using logistic regression
        logreg.fit(trainingArr,trainingLabels.ravel())
        
        #prediction on the training set
        y_pred_train=logreg.predict(trainingArr)
        #prediction on the test set
        y_pred_test=logreg.predict(testingArr)
        
        nTrainCorrect = 0        
        for j in range(0,nTrain):
            #if predicted value is equal to the actual value increase the count
            if y_pred_train[j] == trainingLabels[j]:
                nTrainCorrect= nTrainCorrect + 1
        #storing the prediction accuracy         
        trainScores[i] = nTrainCorrect/nTrain
        
        nTestCorrect = 0        
        for j in range(0,nTest):
            #if predicted value is equal to the actual value increase the count
            if y_pred_test[j] == testingLabels[j]:
                nTestCorrect = nTestCorrect +1
        #storing the prediction accuracy
        testScores[i] = nTestCorrect/nTest
    
    #finding mean of training scores for 10 passes
    trainScoresMean = mean(trainScores)
    #finding mean of testing scores for 10 passes
    testScoresMean = mean(testScores)
    return [trainScoresMean, testScoresMean]       

In [24]:
trainScoreMean1, testScoreMean1 = featureSubsetScore(arr, labels)
trainScoreMean2, testScoreMean2 = featureSubsetScore(arr[:,[0,2,4]], labels)
print('trainScoreMean for input [arr, labels] = ', trainScoreMean1)
print('testScoreMean for input [arr, labels] = ', testScoreMean1)
print('trainScoreMean for input [arr[:,[0,2,4]], labels] = ', trainScoreMean2)
print('testScoreMean for input [arr[:,[0,2,4]], labels] = ', testScoreMean2)

trainScoreMean for input [arr, labels] =  0.857
testScoreMean for input [arr, labels] =  0.8271428571428572
trainScoreMean for input [arr[:,[0,2,4]], labels] =  0.7395
testScoreMean for input [arr[:,[0,2,4]], labels] =  0.7
